In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from itertools import product
import matplotlib.font_manager

In [2]:
models_included = ['ET', 'RF', 'GB', 'MLP']
qos_levels = [1.1, 1.2, 1.3]
classes_ = [2, 3]
features = ['sens', 'cont']
metrics_str = ['accuracy', 'f1-score']

In [3]:
def parse_prediction(args):
    (model, feat, classes, qos) = args
    metrics = {'ET': {'sens': {2: {1.1: {'accuracy': 0.875, 'f1-score': 0.865}, 1.2: {'accuracy': 0.902, 'f1-score': 0.865}, 1.3: {'accuracy': 0.905, 'f1-score': 0.800}},
                               3: {1.1: {'accuracy': 0.845, 'f1-score': 0.755}, 1.2: {'accuracy': 0.865, 'f1-score': 0.755}, 1.3: {'accuracy': 0.902, 'f1-score': 0.715}}},
                      'cont': {2: {1.1: {'accuracy': 0.885, 'f1-score': 0.890}, 1.2: {'accuracy': 0.915, 'f1-score': 0.888}, 1.3: {'accuracy': 0.955, 'f1-score': 0.940}},
                               3: {1.1: {'accuracy': 0.820, 'f1-score': 0.815}, 1.2: {'accuracy': 0.895, 'f1-score': 0.825}, 1.3: {'accuracy': 0.945, 'f1-score': 0.885}}}},
               'RF': {'sens': {2: {1.1: {'accuracy': 0.850, 'f1-score': 0.830}, 1.2: {'accuracy': 0.875, 'f1-score': 0.820}, 1.3: {'accuracy': 0.905, 'f1-score': 0.795}},
                               3: {1.1: {'accuracy': 0.820, 'f1-score': 0.725}, 1.2: {'accuracy': 0.850, 'f1-score': 0.690}, 1.3: {'accuracy': 0.902, 'f1-score': 0.600}}},
                      'cont': {2: {1.1: {'accuracy': 0.870, 'f1-score': 0.870}, 1.2: {'accuracy': 0.915, 'f1-score': 0.890}, 1.3: {'accuracy': 0.955, 'f1-score': 0.940}},
                               3: {1.1: {'accuracy': 0.780, 'f1-score': 0.775}, 1.2: {'accuracy': 0.892, 'f1-score': 0.815}, 1.3: {'accuracy': 0.945, 'f1-score': 0.860}}}},
               'GB': {'sens': {2: {1.1: {'accuracy': 0.850, 'f1-score': 0.832}, 1.2: {'accuracy': 0.910, 'f1-score': 0.875}, 1.3: {'accuracy': 0.915, 'f1-score': 0.825}},
                               3: {1.1: {'accuracy': 0.798, 'f1-score': 0.650}, 1.2: {'accuracy': 0.860, 'f1-score': 0.700}, 1.3: {'accuracy': 0.907, 'f1-score': 0.675}}},
                      'cont': {2: {1.1: {'accuracy': 0.865, 'f1-score': 0.865}, 1.2: {'accuracy': 0.920, 'f1-score': 0.895}, 1.3: {'accuracy': 0.955, 'f1-score': 0.938}},
                               3: {1.1: {'accuracy': 0.780, 'f1-score': 0.778}, 1.2: {'accuracy': 0.890, 'f1-score': 0.815}, 1.3: {'accuracy': 0.935, 'f1-score': 0.810}}}},
               'MLP':{'sens': {2: {1.1: {'accuracy': 0.828, 'f1-score': 0.810}, 1.2: {'accuracy': 0.895, 'f1-score': 0.860}, 1.3: {'accuracy': 0.902, 'f1-score': 0.807}},
                               3: {1.1: {'accuracy': 0.789, 'f1-score': 0.670}, 1.2: {'accuracy': 0.850, 'f1-score': 0.700}, 1.3: {'accuracy': 0.880, 'f1-score': 0.630}}},
                      'cont': {2: {1.1: {'accuracy': 0.855, 'f1-score': 0.860}, 1.2: {'accuracy': 0.930, 'f1-score': 0.908}, 1.3: {'accuracy': 0.960, 'f1-score': 0.950}},
                               3: {1.1: {'accuracy': 0.795, 'f1-score': 0.785}, 1.2: {'accuracy': 0.906, 'f1-score': 0.860}, 1.3: {'accuracy': 0.940, 'f1-score': 0.825}}}}}
    return {'accuracy': metrics[model][feat][classes][qos]['accuracy'], 'f1-score': metrics[model][feat][classes][qos]['f1-score']}

In [4]:
def get_metrics(feat, metric):
    def string(c): return f"{c[1]}{'C' if c[0] == 2 else 'M'}"
    d = {'SLO': map(string, list(product(*[classes_, qos_levels])))}
    for model in models_included:
        d[model] = [parse_prediction([model, feat, c, qos])[metric] for (c, qos) in list(product(*[classes_, qos_levels]))]
    return d

In [5]:
def bar_graphs(feat, metric):
    df = pd.DataFrame.from_dict(get_metrics(feat, metric))
    df = df.set_index('SLO')

    colors  = ['rosybrown', 'silver', 'steelblue', 'goldenrod']
    hatches = len(df) * ['-'] + len(df) * ['/'] + len(df) * [''] + len(df) * ['\\']
    labels = list(map(str, qos_levels)) * 2 
    fontsize = 35
    fname = "Times New Roman"

    ax = df.plot.bar(color = colors, figsize = (15,10), ylim=(0.5, 1.0), rot = 0, zorder = 3, legend = False)
    ax.grid(axis='y', zorder = 0)
    ax.set_xticklabels(labels)
    for bar, hatch in zip(ax.patches, hatches):
        bar.set_hatch(hatch)
        bar.set_edgecolor('black')
    for tick in ax.xaxis.get_major_ticks(): 
        tick.label1.set_fontsize(fontsize)
        tick.label1.set_fontname(fname)
    for tick in ax.yaxis.get_major_ticks():
        tick.label1.set_fontsize(fontsize)
        tick.label1.set_fontname(fname)

    ax.legend(bbox_to_anchor = (0, 1, 1, 0), loc='lower center', frameon=False, ncol = 4, prop = {'family':fname, 'size':fontsize - 5})
    class_definitions = ['Conservative Classification', 'Moderate Classification']
    ax.yaxis.set_label_coords(-0.08, 0.5)
    ax.xaxis.set_label_coords(0.5, -0.15)
    for (i, cd) in enumerate(class_definitions): plt.text(3 * i + 1, 0.425, cd, fontsize = fontsize, ha='center', fontname = fname)
    plt.xlabel("SLO", fontsize = fontsize, fontname = fname)
    plt.ylabel(metric.capitalize(), fontsize = fontsize, fontname = fname)
    plt.tight_layout()
    plt.savefig(f"{feat}_{metric}.png")
    plt.close()

In [6]:
def accuracy_metrics(): 
    for (feat, metric) in list(product(*[features, metrics_str])): bar_graphs(feat, metric)

In [7]:
accuracy_metrics()